# Webapp: set up data stream, visualize, and equitize

- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
- https://dash.plotly.com/live-updates
- https://pbpython.com/interactive-dashboards.html#id6
- https://mybinder.org/ + https://github.com/echow1/trading_music
- (maybe) https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/
- https://kapernikov.com/ipywidgets-with-matplotlib/

In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os, time, re
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PLOT_WIDTH = 1200
PLOT_HEIGHT = 800

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

### Setup: read in the music stream

In [2]:
""" Set up music stream """
def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

def nextChunk(csvStream, chunksize=5):
    return(csvStream.get_chunk(chunksize))

# def hasNext(csvStream):
    

INPUT_PATH = "data_processed/maestro/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x)
MUSIC_STREAM_SUBSTR = "maestro_full_music_stream"

music_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if MUSIC_STREAM_SUBSTR in file:
            music_files.append(os.path.join(root, file))

### pick first as the music stream
music_files = sorted(music_files) # play in order
print("Number of music streams found:")
print(len(music_files))
print(music_files[:10])

### should only 1 have file to stream
if (len(music_files) > 1):
    whichStream = int(input("Index (0 ... N-1) of stream to pick:"))
else:
    whichStream = 0
musicStream = csvStream(music_files[whichStream])

Number of music streams found:
1
['data_processed/maestro/maestro_full_music_stream.csv']


### Task 0: Stream music audio and display as a webapp

### Task 1: Stream MIDI and display as a webapp

(do audio later after download)

In [3]:
# import jp_proxy_widget
# from scipy.io import wavfile

""" Demo: auto-updating time series plot, use with voila and watch update """

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

while True:
    currBar = nextChunk(musicStream, chunksize=100)
    if currBar is None:
        print("End of stream!")
        break
    raise Exception()
    time.sleep(10)

Exception: 

In [11]:
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
    dcc.Interval(
        id='interval-component',
        interval=2*1000, # in milliseconds
        n_intervals=0
    )
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input("colorscale-dropdown", "value"),
    Input("interval-component", "n_intervals"))
def update_figure(colorscale, n_intervals):
    currBar = nextChunk(musicStream)
    fig = px.scatter(
        # like ggplot with color based on group
        currBar, x="start_sec", y="pitch_mean", color="pitch_mean",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Average pitch per bar (streaming)",range_y=[21, 108])
    return(fig)

# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [ ]:
import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output

# pip install pyorbital
from pyorbital.orbital import Orbital
satellite = Orbital('TERRA')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    html.Div([
        html.H4('TERRA Satellite Live Feed'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='live-update-graph'),
        dcc.Interval(
            id='interval-component',
            interval=1*1000, # in milliseconds
            n_intervals=0
        )
    ])
)


@app.callback(Output('live-update-text', 'children'),
              Input('interval-component', 'n_intervals'))
def update_metrics(n):
    lon, lat, alt = satellite.get_lonlatalt(datetime.datetime.now())
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.Span('Longitude: {0:.2f}'.format(lon), style=style),
        html.Span('Latitude: {0:.2f}'.format(lat), style=style),
        html.Span('Altitude: {0:0.2f}'.format(alt), style=style)
    ]


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph_live(n):
    satellite = Orbital('TERRA')
    data = {
        'time': [],
        'Latitude': [],
        'Longitude': [],
        'Altitude': []
    }

    # Collect some data
    for i in range(180):
        time = datetime.datetime.now() - datetime.timedelta(seconds=i*20)
        lon, lat, alt = satellite.get_lonlatalt(
            time
        )
        data['Longitude'].append(lon)
        data['Latitude'].append(lat)
        data['Altitude'].append(alt)
        data['time'].append(time)

    # Create the graph with subplots
    fig = plotly.tools.make_subplots(rows=2, cols=1, vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    fig.append_trace({
        'x': data['time'],
        'y': data['Altitude'],
        'name': 'Altitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 1, 1)
    fig.append_trace({
        'x': data['Longitude'],
        'y': data['Latitude'],
        'text': data['time'],
        'name': 'Longitude vs Latitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 2, 1)

    return fig

### Task 2: Play music and visualize primitive information, construct financial equities, visualize

In [ ]:
""" SPECTROGRAM
    No machine learning required for this.
"""

#

In [ ]:
""" NOTE DENSITY
    No machine learning required for this.
"""

#

### Task 3: Extract music features from MIDI in real-time, construct financial equities, visualize

Sequential learning.
- Validate (try out) against the streamed music audio and series.
- These are the constructed underlyings for financial equities, upon which prediction/regression can work.

Make a local API so other scripts can GET/POST requests (bid/ask) for this.

In [ ]:
""" TEMPO
    Strategy: linear Gaussian state space model / Kalman filter.
    Model tempo (latent variable zt) as a function of notes etc. (observed variables x1 ... xt)
    https://www.researchgate.net/publication/224711190_A_Modified_Kalman_Filtering_Approach_to_On-Line_Musical_Beat_Tracking
"""




In [ ]:
""" HARMONY
"""

#

In [ ]:
""" RHYTHM
"""

#